<a href="https://colab.research.google.com/github/BlackHole3344/Video-annotar/blob/main/VideoAnotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import os
from huggingface_hub import snapshot_download

def download_model(model_id  , local_dir : str = "./models") :
  try :
    if not os.path.exists(local_dir) :
      os.makedirs(local_dir)

    model_path = snapshot_download(
        repo_id = model_id ,
        local_dir = local_dir ,
        local_dir_use_symlinks = False
    )

    print(f"model_downloaded to : {model_path}")
  except Exception as e :
    print(f"error downloading model : {str(e)} "
    )

    return None





In [ ]:
download_model("Chat-UniVi/Chat-UniVi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1212: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/75.8k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

model_downloaded to : /content/models


In [ ]:
!pip install decord


In [ ]:
!pip install transformers

In [ ]:
!git clone https://github.com/PKU-YuanGroup/Chat-UniVi

Cloning into 'Chat-UniVi'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 449 (delta 165), reused 121 (delta 121), pack-reused 261 (from 1)
Receiving objects: 100% (449/449), 39.09 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (214/214), done.


ERROR: Could not find a version that satisfies the requirement ChatUniVi (from versions: none)
ERROR: No matching distribution found for ChatUniVi


In [ ]:
import torch
from transformers import AutoModelForCausalLM , AutoTokenizer
from PIL import Image
from typing import List , Tuple , Dict
from decord import VideoReader
import decord
import logging
from pathlib import Path
import pandas as pd
import logger


ModuleNotFoundError: No module named 'logger'

In [ ]:
import sys
sys.path.append("/content/Chat-UniVi")

In [ ]:
import torch
import os
from ChatUniVi.constants import *
from ChatUniVi.conversation import conv_templates, SeparatorStyle
from ChatUniVi.model.builder import load_pretrained_model
from ChatUniVi.utils import disable_torch_init
from ChatUniVi.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
from PIL import Image
from decord import VideoReader, cpu
import numpy as np

In [ ]:
class VideoProcessor :

    def __init__(self ,
                 model_path : str = "./models" ,
                 image_processor = None ,
                 mode : str = "framerate",
                 batch_size : int = 16,
                 max_frames : int = 100,
                 target_fps : float =1 ):

        self.model_path = Path(model_path)
        self.image_processor = image_processor
        self.mode = mode
        self.batch_size = batch_size
        self.max_frames = max_frames
        self.target_fps = target_fps

        logging.basicConfig(level = logging.INFO)
        self.logger = logging.getLogger(__name__)

        self.setup_model()



    def setup_model(self):

      try:
          self.logger.info(f"Loading model from: {self.model_path}")

          # Check if model exists
          if not os.path.exists(self.model_path):
              raise FileNotFoundError(f"Model not found at {self.model_path}")

          # Load the model
          self.model = AutoModelForCausalLM.from_pretrained(
              self.model_path,
              device_map="cuda",  # Correct parameter name
              trust_remote_code=True  # Correct parameter name
          )

          # Load the tokenizer (using AutoProcessor instead of AutoModelForCausalLM)
          self.tokenizer = AutoProcessor.from_pretrained(
              self.model_path,
              trust_remote_code=True
          )

          # Setup vision tower
          vision_tower = self.model.get_vision_tower()
          if not vision_tower.is_loaded:  # is_loaded is a property
              vision_tower.load_model()

          # Get image processor
          self.image_processor = vision_tower.image_processor

          # Check and log device placement
          device = next(self.model.parameters()).device
          self.logger.info(f"Model loaded successfully on {device}")

          # Optional: Move to GPU if available
          if torch.cuda.is_available():
              self.model = self.model.cuda()
              self.logger.info("Model moved to CUDA device")

      except FileNotFoundError as e:
          self.logger.error(f"Model not found: {str(e)}")
          raise

      except AttributeError as e:
          self.logger.error(f"Error accessing model components: {str(e)}")
          raise

      except RuntimeError as e:
          self.logger.error(f"Runtime error during model loading: {str(e)}")
          raise

      except Exception as e:
          self.logger.error(f"Unexpected error loading model: {str(e)}")
          raise





    def get_frames_indices(self , vreader) -> List[int] :

        total_frames = len(vreader)

        if self.mode == "framerates":

            fps = vreader.get_avg_fps()
            stride = int(round(fps / self.target_fps))
            indices = list(range(0 , total_frames , stride))

            return indices[:self.max_frames]
        else :
            sample_rate = max(1 , total_frames // self.max_frames)
            return list(range(0 , total_frames , sample_rate))


    def process_frames(self , frames : torch.tensor , indices : List[int]) -> Dict :

        if self.image_processor :
            processed_frames = []

            if isinstance (frames , torch.tensor) :
                frames = frames.numpy()

            for frame in frames :
                img = Image.fromarray(frame)
                processed = self.image_processor(img , return_tensors = "pt")
                processed_frames.append(processed["pixel_values"][0])
            frames = torch.stack(processed_frames)

        return {

            "frames" : frames ,
            "indices" : indices
        }



    def process_video(self , video_path : str ) -> Tuple[torch.tensor] :

        decord.bridge.set_bridge("torch")
        vreader = VideoReader(video_path , ctx = cpu(0))
        indices = self.get_frames_indices(vreader)

        all_frames = []

        metadata = []

        for i in range(0 , len(indices) , self.batch_size) :
            batch_indices = indices[i : i + self.batch_size]
            frames = vreader.get_batch(batch_indices)

            results = self.process_frames(frames , indices)
            all_frames.append(results["frames"])

            metadata.extend([ {
                'index' : idx ,
                'timestamp' : idx/vreader.get_avg_fps()
            } for idx in results['indices']])

        all_frames = torch.cat(all_frames) if all_frames else torch.tensor([])

        description : str = self.generate_description(all_frames)


        return description , {
            'total_frames' : len(all_frames) ,
            'orignal_fps' : vreader.get_avg_fps() ,
            'metadata' : metadata
        }

    def generate_description(self , frames : torch.tensor , prompt : str = "Describe Whats in the Image") -> str :
      try :

        slice_len = frames.shape[0]
        # Prepare prompt with image tokens
        if self.model.config.mm_use_im_start_end:
          prompt = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN * slice_len + DEFAULT_IM_END_TOKEN + '\n' + prompt

        else:
          prompt = DEFAULT_IMAGE_TOKEN * slice_len + '\n' + prompt

        # Setup conversation template
        conv = conv_templates["simple"].copy()  # or whatever conv_mode you're using
        conv.append_message(conv.roles[0], prompt)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()

        # Tokenize input
        input_ids = tokenizer_image_token(
        prompt,
        self.tokenizer,
        IMAGE_TOKEN_INDEX,
        return_tensors='pt'
        ).unsqueeze(0).cuda()


        # Setup stopping criteria


        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        keywords = [stop_str]
        stopping_criteria = KeywordsStoppingCriteria(keywords, self.tokenizer, input_ids)

        # Generate response
        with torch.inference_mode():
          output_ids = self.model.generate(
            input_ids,
            images=frames.half().cuda(),
            do_sample=True,
            temperature=0.2,
            top_p=0.9,
            num_beams=1,
            output_scores=True,
            return_dict_in_generate=True,
            max_new_tokens=1024,
            use_cache=True,
            stopping_criteria=[stopping_criteria]
          )

        # Process output
        output_ids = output_ids.sequences
        input_token_len = input_ids.shape[1]

        # Check for input/output mismatches
        n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
        if n_diff_input_output > 0:
            self.logger.warning(f'{n_diff_input_output} output_ids are not the same as the input_ids')

        # Decode output
        outputs = self.tokenizer.batch_decode(
            output_ids[:, input_token_len:],
            skip_special_tokens=True
        )[0]


        outputs = outputs.strip()
        if outputs.endswith(stop_str):
          outputs = outputs[:-len(stop_str)]
        outputs = outputs.strip()

        return outputs


      except Exception as e :
        self.logger.error('error generation response')



# class KeywordsStoppingCriteria(StoppingCriteria):
#     def __init__(self, keywords, tokenizer, input_ids):
#         self.keywords = keywords
#         self.keyword_ids = [tokenizer.encode(keyword, add_special_tokens=False)
#                           for keyword in keywords]
#         self.tokenizer = tokenizer
#         self.start_len = input_ids.shape[1]

#     def __call__(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
#         for keyword_id in self.keyword_ids:
#             if self.check_for_keyword(output_ids[0, self.start_len:], keyword_id):
#                 return True
#         return False

#     def check_for_keyword(self, output_ids, keyword_ids):
#         window_size = len(keyword_ids)
#         for i in range(max(0, len(output_ids) - window_size + 1)):
#             if output_ids[i:i+window_size].tolist() == keyword_ids:
#                 return True
#         return False





In [ ]:
!pip install tqdm

In [ ]:
import glob
import tqdm

In [52]:
def process_video(video_folder : str , output_file : str = "results.csv") -> pd.DataFrame :
  try :
    vec = VideoProcessor()

    #listing video files for loader
    videos_list = glob.glob("./videos/*.mp4")
    video_list = [e.split("/")[-1] for e in videos_list]
    results = []

    for video_file in tqdm(video_list , desc = "Processing Videos") :
      try :

        description , metadata = vec.process_video(str(video_file))

        result.append( {
            "filename" : video_file ,
            "description" : description ,
            "frames_Processed" : metadata["total_frames"]
        })
      except Exception as e :
        results.append({
            "filename" : video_file ,
            "error" : str(e)
        })

      df = pd.DataFrame(results)
      df.to_csv(outfile , index = False)

    return df

  except Exception as e :
    print(f"Error processing videos : {e}")
    raise
def process_video(video_folder : str , output_file : str = "results.csv") -> pd.DataFrame :
  try :
    vec = VideoProcessor()

    #listing video files for loader
    videos_list = glob.glob("./videos/*.mp4")
    video_list = [e.split("/")[-1] for e in videos_list]
    results = []

    for video_file in tqdm(video_list , desc = "Processing Videos") :
      try :

        description , metadata = vec.process_video(str(video_file))

        result.append( {
            "filename" : video_file ,
            "description" : description ,
            "frames_Processed" : metadata["total_frames"]
        })
      except Exception as e :
        results.append({
            "filename" : video_file ,
            "error" : str(e)
        })

      df = pd.DataFrame(results)
      df.to_csv(outfile , index = False)

    return df

  except Exception as e :
    print(f"Error processing videos : {e}")
    raise
def process_video(video_folder : str , output_file : str = "results.csv") -> pd.DataFrame :
  try :
    vec = VideoProcessor()

    #listing video files for loader
    videos_list = glob.glob("./videos/*.mp4")
    video_list = [e.split("/")[-1] for e in videos_list]
    results = []

    for video_file in tqdm(video_list , desc = "Processing Videos") :
      try :

        description , metadata = vec.process_video(str(video_file))

        result.append( {
            "filename" : video_file ,
            "description" : description ,
            "frames_Processed" : metadata["total_frames"]
        })
      except Exception as e :
        results.append({
            "filename" : video_file ,
            "error" : str(e)
        })

      df = pd.DataFrame(results)
      df.to_csv(outfile , index = False)

    return df

  except Exception as e :
    print(f"Error processing videos : {e}")
    raise


In [ ]:
df = process_video("./videos" , "results.csv")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ERROR:__main__:Runtime error during model loading: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 1.06 MiB is free. Process 3629 has 14.74 GiB memory in use. Of the allocated memory 14.65 GiB is allocated by PyTorch, and 1.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Error processing videos : CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 1.06 MiB is free. Process 3629 has 14.74 GiB memory in use. Of the allocated memory 14.65 GiB is allocated by PyTorch, and 1.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 1.06 MiB is free. Process 3629 has 14.74 GiB memory in use. Of the allocated memory 14.65 GiB is allocated by PyTorch, and 1.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)